In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import missingno as mno
import os

import warnings
import logging
logging.disable(logging.CRITICAL)
from tqdm.autonotebook import tqdm


from darts import TimeSeries, concatenate
from darts.dataprocessing.transformers import Scaler
from darts.models import TFTModel
from darts.utils.likelihood_models import QuantileRegression

import optuna
from optuna.integration import PyTorchLightningPruningCallback

from darts.dataprocessing.transformers import Scaler
from darts.datasets import AirPassengersDataset
from darts.metrics import smape
from darts.models import TCNModel
from darts.utils.likelihood_models import GaussianLikelihood

import torch
from torch.nn import HuberLoss
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
import pytorch_lightning as pl

from sklearn.model_selection import train_test_split

pd.set_option("display.precision",2)
np.set_printoptions(precision=2, suppress=True)
pd.options.display.float_format = '{:,.2f}'.format


In [ ]:
print(torch.backends.mps.is_available())

In [ ]:
%run ../utils/preprocessing.ipynb

In [ ]:
mps_device = torch.device("mps")

if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    accelerator = 'mps'
else:
    print ("MPS device not found.")
    accelerator = 'cpu'

In [ ]:
FREQ_INT = 5

FREQ_CONVENTION = minute_frequencies_conventions[FREQ_INT]

N_TRIALS = 100
EPOCHS = 5

In [ ]:
stock_ticker = ['NVDA']

stocks_full = []

for ticker in stock_ticker:

    stock = load_stock_data(f'../data/resampled_data/{FREQ_CONVENTION}/{ticker}_resampled_{FREQ_CONVENTION}.csv', FREQ_INT)


    X_y_df = separate(stock)
    splits = split_data(**X_y_df)

    ts_splits = transform_splits_to_time_series(**splits)

    ts_full = transform_to_time_series(**X_y_df)

    scaled_splits_data = scale_splits_data(**ts_splits)

    scaled_full_data = scale_full_data(ts_full['ts_X_full'], ts_full['ts_y_full'], scaled_splits_data['scaler_X'], scaled_splits_data['scaler_y'])

    stocks_full.append({
        "ticker": ticker,
        "stock": stock,
        "splits": splits,
        "ts_splits": ts_splits,
        "ts_full": ts_full,
        "scaled_splits_data": scaled_splits_data,
        "scaled_full_data": scaled_full_data
    })

# Optuna

In [ ]:
def objective(trial):
    batch_size     =    64
    n_epochs       =    5

    out_len = 1
    nr_epochs_val_period = 1
    valwait = 1        


    # Other hyperparameters
    in_len = trial.suggest_int("output_chunk_length", 40, 400, step=10)
    hidden_layers = trial.suggest_int("hidden_layers", 8, 32, step=8)
    lstm_layers = trial.suggest_int("lstm_layers", 1, 3, step=1)
    attention_heads = trial.suggest_int("attention_heads", 2, 8, step=2)
    hidden_continuous_size = trial.suggest_int("hidden_continuous_size", 4, 16, step=4)
    dropout = trial.suggest_float("dropout", 0.0, 0.4)
    lr = trial.suggest_float("lr", 1e-4, 1e-3, log=True)

    

    # callbacks
    pruner = PyTorchLightningPruningCallback(trial, monitor="val_loss")
    early_stopper = EarlyStopping("val_loss", min_delta=1e-4, patience=3, verbose=True)

    callbacks = [pruner, early_stopper]

    pl_trainer_kwargs = {
        "accelerator": "cpu", 
        "devices": 1,
        "callbacks": callbacks,
        'log_every_n_steps': 10
    }

    torch.manual_seed(42)

    model = TFTModel(
        model_name="tft_model",
        n_epochs=n_epochs,
        input_chunk_length=in_len,
        output_chunk_length=out_len,
        hidden_size=hidden_layers,
        lstm_layers=lstm_layers,
        num_attention_heads=attention_heads,
        hidden_continuous_size=hidden_continuous_size,
        batch_size=batch_size,
        dropout=dropout,
        nr_epochs_val_period=valwait,
        #likelihood=QuantileRegression(QUANTILES),
        optimizer_kwargs={"lr": 1e-3},
        loss_fn=MSELoss(),
        pl_trainer_kwargs=pl_trainer_kwargs,
        add_relative_index=True,
        force_reset=True,
        save_checkpoints=True
    )
    
    nvda = next((stock for stock in stocks_full if stock['ticker'] == 'NVDA'), None)

    
    scaled_splits_data      =    nvda['scaled_splits_data']
    scaled_full_data        =    nvda['scaled_full_data']
    ts_splits               =    nvda['ts_splits']

    model.fit(
      series                =    scaled_splits_data['scaled_y_train'],
      past_covariates       =    scaled_splits_data['scaled_X_train'],
      val_series            =    scaled_splits_data['scaled_y_val'],
      val_past_covariates   =    scaled_splits_data['scaled_X_val'],
      verbose               =    False
    )

    model = TFTModel.load_from_checkpoint("tft_model")
    
    predictions = model.predict(
        n=len(scaled_splits_data['scaled_X_test']),
        series=scaled_splits_data['scaled_y_val'],
        past_covariates=scaled_full_data['scaled_X_full'],
        num_samples=1,
        n_jobs=-1,
    )

    
    predictions_unscaled = scaled_splits_data['scaler_y'].inverse_transform(predictions)
    
        
    smapes = smape(predictions_unscaled, ts_splits['ts_y_test'])
    smape_test = np.mean(smapes)

    return smape_test

In [ ]:
def print_callback(study=None, trial=None):
    print('===========================================================================================')
    print(f'Current value: {trial.value}, Current params: {trial.params}')
    print(f'Best study: {study.best_trial.number}: Best value: {study.best_value}, Best params: {study.best_trial.params}')

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=N_TRIALS, callbacks=[print_callback])

In [ ]:
study.best_trials

In [ ]:
study.best_trial.value

In [ ]:
study.best_params